In [ ]:
import neat
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import LabelEncoder
import graphviz
excel_file = "F:\Thesis\salesdaily-kaggle.xlsx"
sheet_name = "Worksheet"
column_name = "M01AB"

df = pd.read_excel(excel_file, sheet_name=sheet_name)
DATA=df.drop(columns=['datum','Year','Hour',])


print(DATA.head(50))
""" plt.plot(DATA['Month'],DATA['M01AB'],'o-')
plt.show() """

In [ ]:
med1data= DATA[['M01AB','Month','Weekday Name']].copy()
label_encoder = LabelEncoder()
med1data['Weekday Name'] = label_encoder.fit_transform(med1data['Weekday Name'])
""" print(med1data.head()) """

for i in range (1,8):
    med1data[f'Lag{i}']=med1data['M01AB'].shift(i)
med1data.dropna(inplace=True)
print(med1data.head(100))


In [ ]:
x=med1data.drop(columns='M01AB')
y=med1data['M01AB']

x_train,x_test,y_train,y_test= train_test_split(x,y,train_size=.8,random_state=42)
""" print(x_train,x_test) """
""" x_test,x_val, y_test, y_val=train_test_split(x_train,y_train,train_size=.5,random_state=42) """
""" print("Shape of x_train:", x_train.shape)
print("Shape of x_test:", x_test.shape)
print(y_test.shape) """

""" print(x_train.info()) """

config_file="F:\Thesis\SP maam code snippets\[NEAT].txt"
config= neat.Config(neat.DefaultGenome,neat.DefaultReproduction,neat.DefaultSpeciesSet,neat.DefaultStagnation,config_file)


In [ ]:

def evaluate_fitness(genomes, config):
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        predictions = []

        # Make predictions on the training data
        for _, input_data in x_train.iterrows():
            prediction = net.activate(input_data.values)
            predictions.append(prediction[0])  # Assuming single output node

        # Calculate fitness score based on predictions and target values using MSE
        mse = np.mean((np.array(predictions) - y_train.values) ** 2)
        fitness = 1 / (1 + mse)
        genome.fitness = fitness



# Create NEAT population
p = neat.Population(config)

# Add reporters for monitoring progress
p.add_reporter(neat.StdOutReporter(True))
p.add_reporter(neat.Checkpointer(5))  # Save checkpoints every 5 generations

# Run NEAT algorithm
num_generations = 100
for i in range(num_generations):
    # Evaluate fitness for the current generation
    winner = p.run(lambda genomes, config: evaluate_fitness(genomes, config), 1)
    valid_genomes = [genome for genome in p.population.values() if genome.fitness is not None]

    # Get the best genome from the current generation
    best_genome = max(valid_genomes, key=lambda x: x.fitness)

# After training, you can use the best genome to make predictions on the test data
best_net = neat.nn.FeedForwardNetwork.create(best_genome, config)
predictions = []

for _, input_data in x_test.iterrows():
    prediction = best_net.activate(input_data.values)
    predictions.append(prediction[0])  # Assuming single output node

# Evaluate the predictions
mse_test = np.mean((np.array(predictions) - y_test.values) ** 2)
print("Mean Squared Error on Test Data:", mse_test)


In [19]:
best_genome_id = 13874
best_genome = None

# Find the best genome from the NEAT algorithm
for genome_id, genome in p.population.items():  # Access the population directly
    if genome_id == best_genome_id:
        best_genome = genome
        break
best_net = neat.nn.FeedForwardNetwork.create(best_genome, config)


In [26]:
import graphviz

def draw_network(network, filename=None, view=False):
    dot = graphviz.Digraph()

    # Add input nodes
    for i in range(network.input_size):
        dot.node(f"I{i}", shape="circle", color="blue")

    # Add hidden nodes
    for i in range(network.input_size, network.input_size + network.hidden_size):
        dot.node(f"H{i}", shape="circle", color="green")

    # Add output nodes
    for i in range(network.input_size + network.hidden_size, network.input_size + network.hidden_size + network.output_size):
        dot.node(f"O{i}", shape="circle", color="red")

    # Add edges
    for node_id, node in network.connections.items():
        if node.enabled:
            in_node, out_node = node.key
            dot.edge(f"{in_node}", f"{out_node}", label=str(round(node.weight, 2)))

    if filename:
        dot.render(filename, format='png', cleanup=True)
    if view:
        dot.view()


draw_network(best_net, config)


AttributeError: 'FeedForwardNetwork' object has no attribute 'input_size'